<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/final/multi_cls_sobert_base_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install datasets>=2.18.0 transformers>=4.38.2 sentence-transformers>=2.5.1 setfit>=1.0.3 accelerate>=0.27.2 seqeval>=1.2.2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
model_path = '/content/drive/My Drive/multi_cls_sobert_base/multi_cls_sobert_base'  # 모델과 토크나이저가 저장된 경로

In [4]:
classes = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']
class2id = {'Algorithms' :0, 'Backend' : 1, 'Data Science' : 2, 'Databases' : 3, 'Dev Tools' : 4, 'Frontend' : 5, 'Mobile' :6, 'Systems' : 7, 'iOS/macOS' : 8}
id2class = {0 : 'Algorithms', 1: 'Backend', 2 : 'Data Science', 3 : 'Databases', 4 : 'Dev Tools', 5 : 'Frontend', 6 : 'Mobile', 7 : 'Systems', 8 :'iOS/macOS'}

In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [10]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MegatronBertForSequenceClassification(
  (bert): MegatronBertModel(
    (embeddings): MegatronBertEmbeddings(
      (word_embeddings): Embedding(50048, 768, padding_idx=0)
      (position_embeddings): Embedding(2048, 768)
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): MegatronBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x MegatronBertLayer(
          (attention): MegatronBertAttention(
            (ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (self): MegatronBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MegatronBertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [27]:
# 예측 함수 정의
def predict(texts, m, tk):
    # 텍스트를 토큰화하고 텐서로 변환
    inputs = tk(texts, padding='max_length', truncation=True, max_length=512, return_tensors='pt').to(m.device)

    # 모델을 사용해 예측 수행
    with torch.no_grad():
        outputs = m(**inputs)
        logits = outputs.logits[0]

    probabilities = torch.sigmoid(logits).cpu().numpy()
    predictions = (probabilities > 0.5).astype(int)
    result = [id2class[i] for i, value in enumerate(predictions) if value == 1]
    return result

In [32]:
# 샘플 텍스트 입력 및 예측 수행
text =  """does laragon have backup folder for MySQL like Xampp or am i not safe?

i switched from XAMPP to laragon because the mysql fails 2 times a week no matter what i do and i keep getting inconsistent errors that suggest systematic corruption of the mysql and despite trying trying to reinstall XAMPP before it seems like no matter what i do the problem arises after a while no matter how long MySQL stays stable it fails after i can drop my error logs if anyone is interested or have any knowledge of this peculiar behavior anyways i don't if it makes a difference but i switched to laragon and it has cleaner interface and felt more simpler however one thing i noticed in the MySQL folder is that there is no backup folder, is this normal?

I tried going into the Laragoon folder and found no backup folder to MySQL
"""


# 예측 수행
print(predict(text, model, tokenizer))

['Backend', 'Databases']
